# Claude 3.5 Sonnet 모델 테스트

In [8]:
pip install boto3 streamlit langchain

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ------------------ --------------------- 5.8/12.7 MB 29.4 MB/s eta 0:00:01
   ------------------------ --------------- 7.9/12.7 MB 21.2 MB/s eta 0:00:01
   ---------------------------- ----------- 9.2/12.7 MB 15.9 MB/s eta 0:00:01
   --------------------------------- ------ 10.5/12.7 MB 12.8 MB/s eta 0:00:01
   ------------------------------------- -- 11.8/12.7 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 12.7/12.7 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   --------------------------- ------------ 1.3/1.9 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 5.6 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.35.52 which is incompatible.


In [66]:
import boto3
import os

# 환경 변수에서 인증 정보 불러오기
session = boto3.Session(
    region_name="us-east-1"
)

# Bedrock 클라이언트 생성
bedrock = session.client("bedrock")

# Claude 3.5 Sonnet 모델에 대한 프롬프트 정의
prompt_text = "현재 대한민국 날씨를 알려줘."

try:
    # API 호출
    response = bedrock.invoke_model(
        modelId="claude-3.5-sonnet",
        contentType="text/plain",
        accept="application/json",
        body=prompt_text
    )
    
    # 응답 출력
    result = response["body"].read().decode("utf-8")
    print("모델 응답:", result)

except Exception as e:
    print("오류 발생:", e)


오류 발생: 'Bedrock' object has no attribute 'invoke_model'


In [68]:
import boto3
import json

# AWS 서비스 초기화
session = boto3.Session(
    aws_access_key_id="tmp",  # AWS Access Key 입력
    aws_secret_access_key="tmp",  # AWS Secret Key 입력
    region_name="us-east-1" # Claude 3.5 Sonnet이 지원되는 리전으로 설정
)

# Bedrock 클라이언트 생성
bedrock = session.client("bedrock")

# Claude 3.5 Sonnet 모델에 대한 프롬프트 정의
prompt_data = {
    "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": "현재 대한민국 날씨를 알려줘."
            }
        ]
    }
}

try:
    # API 호출
    response = bedrock.invoke_model(
        modelId=prompt_data["modelId"],
        contentType=prompt_data["contentType"],
        accept=prompt_data["accept"],
        body=json.dumps(prompt_data["body"])
    )
    
    # 응답 출력
    result = response["body"].read().decode("utf-8")
    print("모델 응답:", result)

except Exception as e:
    print("오류 발생:", e)

오류 발생: 'Bedrock' object has no attribute 'invoke_model'


In [70]:
import requests
import json

# API 엔드포인트 및 인증 정보 설정
endpoint = "https://bedrock.us-east-1.amazonaws.com/invoke_model"
api_key = "dnS7myFSC4CLAD5ok5nI+r9fQRjqyXPx0FaS+6z6"  # API 키를 환경 변수나 AWS Secrets Manager에서 안전하게 관리하세요.

# 헤더 및 데이터
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
    "Accept": "application/json"
}

prompt_data = {
    "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
    "body": {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": "현재 대한민국 날씨를 알려줘."
            }
        ]
    }
}

try:
    # API 호출
    response = requests.post(endpoint, headers=headers, json=prompt_data)
    
    # 응답 출력
    if response.status_code == 200:
        result = response.json()
        print("모델 응답:", result)
    else:
        print("오류 발생:", response.text)

except Exception as e:
    print("오류 발생:", e)

오류 발생: 'Bedrock' object has no attribute 'invoke_model'


In [35]:
import boto3
import json
from botocore.config import Config
from dotenv import load_dotenv
import os

# .env 파일 로드 (AWS 자격 증명 및 리전 정보)
load_dotenv()

# AWS 자격 증명 설정
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name='ap-northeast-2'
)

# 클라이언트 구성 (재시도 설정 포함)
config = Config(retries={"max_attempts": 3, "mode": "adaptive"})
client = session.client("bedrock-runtime", config=config)

# Claude Sonnet 모델 설정
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Sonnet 모델에 요청하는 함수 정의
def generate_sonnet_response(input_text, max_tokens=1000):
    # 단일 제목 생성 요청을 강조하며 한국어로 요청
    request_suffix = "\n\n위 작업을 요약하여 하나의 간단한 제목을 한국어로 작성해"
    full_input_text = f"{input_text.strip()}{request_suffix}"
    
    # 요청 메시지 구성
    messages = [
        {
            "role": "user",
            "content": full_input_text
        }
    ]
    
    payload = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": messages
    }
    
    try:
        print("Sonnet 모델 요청 중...")
        
        # 모델 호출
        response = client.invoke_model(
            modelId=model_id,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(payload)
        )

        # 응답 결과 파싱 및 출력
        result = json.loads(response['body'].read().decode("utf-8"))
        assistant_response = result.get('content', [{}])[0].get('text', "응답을 확인할 수 없습니다.")
        
        return assistant_response  # 응답을 반환하고 바로 출력하지 않음

    except client.exceptions.ThrottlingException:
        print("요청이 너무 많습니다. 잠시 후 다시 시도하세요.")
        
    except Exception as e:
        print("Sonnet 모델 호출 중 오류 발생:", e)


In [37]:
# 예시 질문 입력 셀
input_text = """
1. 수행 작업
    1. AWS FM model 적용
    2. Titan v2.0
    3. Claude 3.5 Sonnet
    4. Claude 3 Haiku

2. 특이 사항
    1. 지역에 따른 서비스 불가 가능성 有
    2. Claude 3.5 Sonnet 모델의 쓰로틀링 발생
    3. 인가 관련 설정 多

3. 향후 계획
    1. 개발일지 관련 자동화 로직 구현
    2. 사용자 피드백 반영
"""
response = generate_sonnet_response(input_text)
print("Sonnet 모델의 응답:", response)

Sonnet 모델 요청 중...
Sonnet 모델의 응답: "AWS와 Claude AI 모델 적용 및 개선 계획"

이 제목은 주요 작업인 AWS와 Claude AI 모델 적용을 언급하면서, 특이 사항과 향후 계획을 포괄적으로 아우르는 내용을 간단하게 요약했습니다.
